In [ ]:
import requests
import spotipy
import time
import sys
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from dataprep.eda import plot, plot_correlation, plot_missing
from datetime import datetime

time: 2.53 ms (started: 2021-03-20 20:01:46 +00:00)


In [ ]:
!pip3 install spotipy

In [ ]:
!pip install dataprep

In [ ]:
!pip install ipython-autotime

In [ ]:
%load_ext autotime

time: 100 µs (started: 2021-03-20 20:01:17 +00:00)


In [ ]:
#Put this function in front of the 'chunk' cells later (or any cells you
# want to time while they are running).  You don't need to pass anything
# into it.

def st_time():
  now=datetime.now()
  st_time=now.strftime("%H:%M:%S")
  print('Start time:', st_time)
  return

time: 2.9 ms (started: 2021-03-20 20:01:20 +00:00)


# Get authorization from Spotify to start the data vaccuum

**This is the method that is able to 'talk' to the Spotify API.**

There are 4 required fields for a broad session.
1) 'cliend_id' which is just the id number of your own account.  It can likly be found a few ways, but the way I did it was:
* Open the spotify app
* Go to the upper right corner
* Select the profile icon
* Select '...' under username
* Select "Share"
* Select "Copy Spotify URI"
* Paste in a new cell, which yields: spotify:user:user_id
* Select only the alphanumeric user_id.  

[Note: The 'Copy Spotify URI' is also how you can grab other user_id for later]  

Now we will go to the Spotify Developer page so you can see the 'client_secret'.

2) 'client_secret'.  Go to https://developer.spotify.com/dashboard --> log in 
and select through whatever kind of use case you need.  I selected making an
app.  Your selection doesn't really matter at this stage as you can use the
code you are building here and start a new app that fits more precisely your
use case.

You are now here: https://developer.spotify.com/dashboard/applications
*  Select your app.  
* On the left side of the page you will see 'Client ID'.  Your personal ID (read: 'user_id') is a long alphanumeric string.
* Click on 'SHOW CLIENT SECRET' to access your 'client_secret'

3) 'redirect_uri'.  As best as I can tell, 'redirect_uri' is something along the lines of a browser page Spotify needs for you to have so they can use it as an address to ask authorization questions.  I read you can use basically any 'white listed' website (a normal website...like https://google.com) or a local host like "http://localhost:8890/callback", which is the format I chose.  Here, the number '8880' is the default one.  I just pick a higher number that likely won't be in use like 8901 or something.

4) 'scope'.  This will vary depending on what you are trying to do.  For this project, I am sticking to 'read' only scopes.  But you need different scopes for authorization to do different things so look at the offical documentation, or better yet, someone elses expalanation of the documentation to understand what the scope of your own project is.


In [ ]:
#This is the method that is able to 'talk' to the Spotify API.
#There are 4 required fields for a broad session: 1) 'client_id'
# 'client_secret', 'redirect_uri', 'scope'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="REDACTED",
                                               client_secret="REDACTED",
                                               redirect_uri="https://vasovagalband.com/",
                                               scope="user-library-read"))

time: 3.02 ms (started: 2021-03-20 20:01:20 +00:00)


In [ ]:
spotify_pl = sp.user_playlists('spotify')

User authentication requires interaction with your web browser. Once you enter your credentials and give authorization, you will be redirected to a url.  Paste that url you were directed to to complete the authorization.
Opened https://accounts.spotify.com/authorize?client_id=39ded50cd91a464492c532e4785b1d05&response_type=code&redirect_uri=https%3A%2F%2Fvasovagalband.com%2F&scope=user-library-read in your browser


Enter the URL you were redirected to: https://vasovagalband.com/?code=AQDdZ1cgDhr-tlv0K7N06R9xNPmWd1KX4B12sx22j4dzitt8xDzp5BusL8YshQadVyLZObBPB1Rj2U5IB9oUhTsHhCdt9SW6MDwIpAvFOR59-8pZouqAWNHoUHTsADISeUE6ZG2LYxAtipSWvVhqVfkuYkZDqcnvDDkNt620M8Z5jplmLToTeR3ZFPuPGkAqkA
time: 12.3 s (started: 2021-03-20 20:01:20 +00:00)


If you get 200-500 errors check the Spotify documentation to understand the cause of error.  Otherwise, it shoud take <1s to authentify.

# Collect Playlists from User

We need playlists to build a song library and one of the best ways to do that is to find users with a lot of playlists.  I wrote a function that will take a user_id and give you a list of playlists.

Lets start with the official Spotify account since it will likely have a lot of playlists without broken songs.  And it will have songs that beta tests will at least recognize to help with the content filtering.

### The official spotify user account playlists

spotify_pl = sp.user_playlists('spotify')

[NOTE: Normally user_id is alphanumeric nonsense but the official Spotify account has its name as its ID]

In [ ]:
#Function I wrote to get playlist IDs from a single user.
# The input is a bunch of JSON dict-list nightmare nonsense.
# The output is a list of playlists for input into 'getTrackIDs'
# function.

#Normally the playlist output length is capped at 50.  So the
# function below is written to paginate and grab all of them.
# The 'while' loop is how paging is accomplished.

#NOTE: to call the next method ("sp") it takes the dictionary object "results"

def pl_grab(user_id):
    results = sp.user_playlists(user_id)    #Request function for url.  The 'results' are the data taken from the request method 'sp'.
    pl_id = []                              #This is the empty list we are going to populate with just the playlist ids.
    pl = results['items']                   #take results from the sp request and add only 'items' to variable pl. There are 14 keys/divisions(keys after you make it into a dataframe) of 'items'.
    while results['next']:                  #this while loop will create a list of dictionaries. "while results['next']" is a boolean so that if the parameter 'next' is populated (true) then it will continue running the while loop.
        results = sp.next(results)          #this is like the api request method the second line of the function except it runs the url from the 'next' key/divsion/category? There was only 1 'next' populated for all the first 50 results originally.  So when you got to a playlist that also had the parameter 'next' populated it meant it was the last one on the page and that there was at least one more page to go to.
        pl.extend(results['items'])         #extend is like append.  Here we are adding the 'items' data that we will pull 'id' from to the previous selected results 'pl'.  Note, this just adds to the original pl and not overwrite. This is how we build them all up.
    for playlist_dict in pl:                #NOTE: type(pl) = list, type(playlist_dict) = dict.  This means that the var 'pl' is a list and the elements are in the form of a dictionary so we need to use the dict syntax (square brackets and quotes) in the foreloop to proerly index the 'id's.  This was also a hang up because this is a results of the JSON-to-dataframe 'parsing'.  But hey, it works and actually isn't that messy. loops through dictionaries and only grabs ids. This is where i got stuck earlier and had the for looop nested in the while loop.  It needed to be out so that it only ran after the dataframe was finished being created from all of the possible playlists on all pages.
        pl_id.append(playlist_dict['id'])   #NOTE: as in the line above, just see that playlist_dict is the iterate and is a type=dict within the greater type=list. this adds the playlist 'id's to the list of ids ('pl_id')
    return pl_id                            #the pl_id format is just the ID's and not all the JSON nonsense

time: 6.56 ms (started: 2021-03-20 20:01:59 +00:00)


In [ ]:
#Grab the playlists from user 'spotify'

pl_spotify = pl_grab('spotify')

time: 10.1 s (started: 2021-03-20 20:02:13 +00:00)


In [ ]:
#Now let's see how many playlists there are.

print("There are", len(pl_spotify), "playlists.")

There are 1398 playlists.
time: 1.58 ms (started: 2021-03-20 20:02:23 +00:00)


# Collect Tracks from Playlists

We now have a *list of playlists*.  We want to get a list of tracks.  First, we will end up with a *list of a list of tracks*.

This is the same general process as above for taking playlists from a user, but instead we extract 'track_id' from playlists.

In [ ]:
#returns a list of specific track IDs from a spotify playlist and creator userid

def tr_grab(user, playlist_id):
    track_ids = []
    results = sp.user_playlist_tracks(user, playlist_id) #NOTE: type(playlist) = dict but it's actually a dict with nested lists and dicts.  the JSON nonsense pile.
    playlists = results['items']                         #This is the JSON nonsense that carries all the 'keys' and such for ALL the playlists,  you had to do differnt JSON navigating to get to the right part. The key 'items' is within results and type(results) = dict.  The key 'track' is within items and type(items) = dict. The division 'id' is within 'track' and type(track) = list.
    while results['next']:                               #this 'while' loop is the same as the 'grab_pl' function in that it is a boolean.  We had to navigate down to where 'next' was populated with a url (meaning there are more pages of songs to go to).
        results = sp.next(results)                       #This will call the request method 'sp' so you can get another page of results.  The 'next' is where the url is.
        playlists.extend(results['items'])               #extend is like append but slightly different.
    for item in playlists:                               #loop through and find just the 'id' and add to list 'track_ids'.
        track = item['track']
        track_ids.append(track['id'])
    return track_ids


time: 7.6 ms (started: 2021-03-20 20:02:03 +00:00)


In [ ]:
#This function will call the 'tr_grab' function as part of a try-except nested
# in a for loop. This is necessary function skips broken playlists, which will
# break the tr_grab function.

##Inputs: 'user_id' (same one you've been using), 'pl_list' which is the list
## of playlists.

##Output: 'pl_tr_list', which is a list where elements = tracks of a single
## playlist.

def tr_from_pl(user_id,pl_list):
    pl_tr_list=[]
    for playlist in pl_list:
        try:    #gave back NoneType without the try-except
            tr_id = tr_grab(user_id,playlist)
            pl_tr_list.append(tr_id)
        except:
            pass
    return pl_tr_list

time: 4.83 ms (started: 2021-03-20 20:02:25 +00:00)


In [ ]:
##This cell calls the 'tr_from_pl' function, which loops through the 'tr_grab'
# function.

#Note: Make sure to assign the function call to a varible before running.

#Execute time was 6m5.09s for 102,919 songs.

tot_track_list = tr_from_pl('37i9dQZF1DXcBWIGoYBM5M',pl_spotify)

time: 2min 52s (started: 2021-03-20 20:02:25 +00:00)


In [ ]:
#This shows the number of lists of tracks.  It is equivalent to the number
# of playlists that satisfied the 'try' statement in 'tr_from_pl'

print("The number of track lists is",len(tot_track_list))
print("The number of broken playlists is", len(pl_spotify)-len(tot_track_list))
print("The percentage of working playlists is", (len(tot_track_list)/len(pl_spotify))*100,"%")

The number of track lists is 1392
The number of broken playlists is 6
The percentage of working playlists is 99.57081545064378 %
time: 2.74 ms (started: 2021-03-20 20:07:20 +00:00)


## Flatten the List-of-list-of-tracks

In [ ]:
#This list comprehension will 'flatten' the output list-of-lists-of-tracks
# from tr_from_pl into a singular list of tracks.  In other words, it makes
# one big list from a list of lists.

tr_entry_tot = [item for sublist in tot_track_list for item in sublist]

time: 13.3 ms (started: 2021-03-20 20:07:22 +00:00)


In [ ]:
#This shows the number of track entries tracks.

print("The number of track entires from 'user_id' is", len(tr_entry_tot))

The number of track entires from 'user_id' is 103323
time: 1.23 ms (started: 2021-03-20 20:07:23 +00:00)


## Get rid of duplicates

In [ ]:
#Get rid of duplicates tracks that appear on more than 1 playlist.

tr_unique_total = list(set(tr_entry_tot))
print("The number of track entries from 'user_id' is", len(tr_entry_tot))
print("The number of unique tracks is",len(tr_unique_total))
print("The percentage of unique tracks relative to track entries is", (len(tr_unique_total)/len(tr_entry_tot))*100,"%")

The number of track entries from 'user_id' is 103323
The number of unique tracks is 82106
The percentage of unique tracks relative to track entries is 79.46536589142785 %
time: 35.8 ms (started: 2021-03-20 20:07:24 +00:00)


In [ ]:
#Test the list to make sure the output is just a list of track_id's.

tr_unique_total

# Get track audio features

In [ ]:
#This function will grab all of the track data.
#This includes meta data for the album and artists. Some of the metadata
# will be important later on.  Specifically, the external URLs for track,
# album, artist, preview, and album art.  The 'href' features may not be
# super important, but I wanted to grab all the available data I could at
# once and filter later since it shouldn't really affect exucution time.


def tr_af(track_id):
    #returns a list of desired track audio features from track ID
    
    track_info = sp.track(track_id)
    features_info = sp.audio_features(track_id)

#track metadata
    track_id = track_info['id'] #redundant, but easier to keep track of this way.
    track_name = track_info['name']
    track_length = track_info['duration_ms']
    track_preview_url = track_info['preview_url']
    track_number = track_info['track_number']
    track_href = track_info['href']
    track_external_urls = track_info['external_urls']['spotify']
    track_external_id_isrc = track_info['external_ids']['isrc']

    album_id = track_info['album']['id']
    album_name = track_info['album']['name']
    album_href = track_info['album']['href']
    album_ext_url = track_info['album']['external_urls']['spotify']
    album_release_date = track_info['album']['release_date']
    album_release_date_precision = track_info['album']['release_date_precision']
    album_total_tracks = track_info['album']['total_tracks']
    album_type = track_info['album']['type']
    album_image_large = track_info['album']['images'][0]['url']
    album_image_medium = track_info['album']['images'][1]['url']
    album_image_small = track_info['album']['images'][2]['url']
    
    artist_id = track_info['album']['artists'][0]['id']
    artist_name = track_info['album']['artists'][0]['name']
    artist_href = track_info['album']['artists'][0]['href']
    artists_ext_url = track_info['album']['artists'][0]['external_urls']['spotify']
    
#Spotify proprietary audio features
    danceability = features_info[0]['danceability']
    energy = features_info[0]['energy']
    key = features_info[0]['key']
    loudness = features_info[0]['loudness']
    mode = features_info[0]['mode']
    speechiness = features_info[0]['speechiness']
    acousticness = features_info[0]['acousticness']
    instrumentalness = features_info[0]['instrumentalness']
    liveness = features_info[0]['liveness']
    popularity = track_info['popularity']
    valence = features_info[0]['valence']
    tempo = features_info[0]['tempo']
    time_signature = features_info[0]['time_signature']

#Note that 'track_data' is sort of a loose term as the info below
# will include album and artist info as well.
    track_data = [track_id, track_name, album_id, album_name,
                  artist_id, artist_name, track_preview_url,
                  track_number, track_href, track_external_urls,
                  track_external_id_isrc, album_href, album_ext_url,
                  album_release_date, album_release_date_precision,
                  album_total_tracks, album_type, album_image_large,
                  album_image_medium, album_image_small, artist_href,
                  artists_ext_url, acousticness, danceability, energy,
                  instrumentalness, key, liveness, loudness, mode,
                  popularity, speechiness, tempo, time_signature,
                  track_length, valence]
    
    return track_data

time: 58.6 ms (started: 2021-03-20 20:07:26 +00:00)


In [ ]:
#Function test area

tr_af_test = tr_af('09u4feGdXwYsvH7hWObLPw')
tr_af_test

In [ ]:
#JSON parsing test area

yeet = sp.track('6CVU1Ckw8SG6sSs5sqGta6')
artist_id = track_info['album']['artists'][0]['external_urls']['spotify']
artist_id

## Two acquistition approaches

### 1) All tracks in a single function call

In [ ]:
#get the df with all the content stuff from Spotify
#got 33s for sleep=0.1 and 20s for no sleep
#'af_df' = 'audio features dataframe'

def af_df(track_ids):
    #returns pandas dataframe of track features from a list of track IDs
    track_data_list = []
    for i in range(len(track_ids)):
        time.sleep(.1)
        try:
            track_data = tr_af(track_ids[i])
            track_data_list.append(track_data)
        except:
            pass

        df =  pd.DataFrame(track_data_list,
                           columns = ['track_id', 'track_name', 'album_id', 'album_name',
                                      'artist_id', 'artist_name', 'track_preview_url',
                                      'track_number', 'track_href', 'track_external_urls',
                                      'track_external_id_isrc', 'album_href', 'album_ext_url',
                                      'album_release_date', 'album_release_date_precision',
                                      'album_total_tracks', 'album_type', 'album_image_large',
                                      'album_image_medium', 'album_image_small', 'artist_href',
                                      'artists_ext_url', 'acousticness', 'danceability', 'energy',
                                      'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
                                      'popularity', 'speechiness', 'tempo', 'time_signature',
                                      'track_length', 'valence'])
    return df

time: 11.1 ms (started: 2021-03-20 20:07:30 +00:00)


## Single Block

In [ ]:
total_df = af_df(tr_unique_total)
total_df

In [ ]:
total_df

### 2) Tracks evenly split for 'chunked' audio feature aquisition

Try the code out on smaller chunks...5-100.

getTrackFeatures will return list of charecteristics.
- write function that takes getTrackFeatures, loops throughs through the items, and creates a dataframe.
- do forlooop for song in 100k songs
    - append
    - turn list into dataframe
    - return dataframe
ASSIGN THE RESULTS TO A VARIABLE

In [ ]:
#Chunks for feature extraction

tr_unique_total
ch_test = tr_unique_total[0:100]
ch_1 = tr_unique_total[0:9999]
ch_2 = tr_unique_total[10000:19999]
ch_3 = tr_unique_total[19999:29999]
ch_4 = tr_unique_total[30000:39999]
ch_5 = tr_unique_total[40000:49999]
ch_6 = tr_unique_total[50000:59999]
ch_7 = tr_unique_total[60000:69999]
ch_8 = tr_unique_total[70000:82189]

time: 7.82 ms (started: 2021-03-20 20:08:38 +00:00)


# Chunk Test

In [ ]:
#Chunk Test.  Estimated execute time: a few minutes

ch_test_df = af_df(ch_test)
ch_test_df

In [ ]:
ch_test

#### Success rate function

In [ ]:
#Function to show success rate of audio feature extraction.
# This is because some songs will not be able to be located and
# cause a 404 error.  And for reasons unknown to me, other songs
# will have a finite number of retries that will sometimes fail.

def chunk_df_eval(chunk,chunk_df):
    print('For chunk_df the dateframe rows,numbers =',chunk_df.shape)
    print('For chunk_df the percentage of successful audio feature extraction is',round(len(chunk_df['track_name'])/len(chunk)*100,2))

time: 2.41 ms (started: 2021-03-20 20:53:32 +00:00)


# Chunking Instructions

1) Grab Tracks

2) Run Evaluation

3) Aggregate Progress

4) Write Progress to a CSV

### Add custom tracks,albums,artists,playlists here

Note: Must be in track_id form.

In [ ]:
#Adding my own band's album to act as tracers.
# Also, because duh, you gotta market yourself somehow.

vaso_vagal=['57lYOdic1C2GVyTPZsJhlB',
            '3Rv7s6zTx6KTYZfrpBdW4C',
            '0Qj2eDY1Nku7OGymMs8HGz',
            '2pgLRrQEvDwd6jTXGDswQm',
            '5SzbfdSfxmmlyTEZcpb3EH',
            '6wUEUO6bE1R84LAB7TIAIs',
            '09u4feGdXwYsvH7hWObLPw',
            '0q4ADFP0VAFpTKyAnWuU0e',
            '7mxdqQjhmmv1iPdrE6VEmB']

time: 1.99 ms (started: 2021-03-20 20:12:06 +00:00)


## Chunk 1/8

In [ ]:
#Chunk 1/8.  Estimated execute time: 1h on local drive. 30m on Google Colab.

st_time()
ch_1_df = af_df(ch_1)

Start time: 23:13:32


HTTP Error for GET to https://api.spotify.com/v1/tracks/77iBDvb6t3P6o4uG2rzQ4b returned 404 due to non existing id


time: 34min 52s (started: 2021-03-20 23:13:32 +00:00)


In [ ]:
#Success evaluation

st_time()
chunk_df_eval(ch_1,ch_1_df)

Start time: 23:51:54
For chunk_df the dateframe rows,numbers = (9970, 36)
For chunk_df the percentage of successful audio feature extraction is 99.71
time: 2.87 ms (started: 2021-03-20 23:51:54 +00:00)


In [ ]:
#Write progress to a csv so you don't lose data with runtime crash.
#'ss' = 'subsample' and corresponds to the aggregae as you go.  It will
# change to 'tot' = "total" after the last chunk is added.

#'af' = audio features, 'tot' = total, 'df' = dataframe

#Band name is "Vaso Vagal"
vaso_vagal_df = af_df(vaso_vagal)
af_tot_df_ss = pd.concat([ch_1_df]+
                   [vaso_vagal_df]).reset_index()
af_tot_df_ss.drop(columns='index')

#Export as a csv.
af_tot_df_ss.to_csv('/content/drive/MyDrive/Project5/af_tot_df_ss.csv')

## Chunk 2/8

In [ ]:
#Chunk 2/8.  Estimated execute time: 1h on local drive. 30m on Google Colab.

ch_2_df = af_df(ch_2)

HTTP Error for GET to https://api.spotify.com/v1/tracks/6Pf5ioiO4SALW4wNehAtW8 returned 404 due to non existing id


time: 35min 15s (started: 2021-03-20 23:51:57 +00:00)


In [ ]:
#Success evaluation

st_time()
chunk_df_eval(ch_2,ch_2_df)

Start time: 00:27:54
For chunk_df the dateframe rows,numbers = (9970, 36)
For chunk_df the percentage of successful audio feature extraction is 99.71
time: 2.53 ms (started: 2021-03-21 00:27:54 +00:00)


In [ ]:
#Write progress to a csv so you don't lose data with runtime crash.
#'ss' = 'subsample' and corresponds to the aggregae as you go.  It will
# change to 'tot' = "total" after the last chunk is added.

#'af' = audio features, 'tot' = total, 'df' = dataframe

#Band name is "Vaso Vagal"
vaso_vagal_df = af_df(vaso_vagal)
af_tot_df_ss = pd.concat([ch_1_df]+[ch_2_df]+
                   [vaso_vagal_df]).reset_index()
af_tot_df_ss.drop(columns='index')

#Export as a csv.
af_tot_df_ss.to_csv('/content/drive/MyDrive/Project5/af_tot_df_ss.csv')

## Chunk 3/8

In [ ]:
#Chunk 3/8.  Estimated execute time: 1h on local drive. 30m on Google Colab.

ch_3_df = af_df(ch_3)

HTTP Error for GET to https://api.spotify.com/v1/tracks/1k2OdSPpAzf7p9ABt7UM0z returned 404 due to non existing id
HTTP Error for GET to https://api.spotify.com/v1/tracks/7wDTsfMlz4Uh6d6xd6Gd7W returned 404 due to non existing id
HTTP Error for GET to https://api.spotify.com/v1/tracks/2LZ515e2Oqyab3WBKr5CQk returned 404 due to non existing id
HTTP Error for GET to https://api.spotify.com/v1/tracks/1FzdxfMvZ6a1SjmDBYFOzN returned 404 due to non existing id


time: 35min 6s (started: 2021-03-21 00:28:00 +00:00)


In [ ]:
#Success evaluation

st_time()
chunk_df_eval(ch_3,ch_3_df)

Start time: 01:07:15
For chunk_df the dateframe rows,numbers = (9962, 36)
For chunk_df the percentage of successful audio feature extraction is 99.62
time: 3.07 ms (started: 2021-03-21 01:07:15 +00:00)


In [ ]:
#Write progress to a csv so you don't lose data with runtime crash.
#'ss' = 'subsample' and corresponds to the aggregae as you go.  It will
# change to 'tot' = "total" after the last chunk is added.

#'af' = audio features, 'tot' = total, 'df' = dataframe

#Band name is "Vaso Vagal"
vaso_vagal_df = af_df(vaso_vagal)
af_tot_df_ss = pd.concat([ch_1_df]+[ch_2_df]+[ch_3_df]+
                   [vaso_vagal_df]).reset_index()
af_tot_df_ss.drop(columns='index')

#Export as a csv.
af_tot_df_ss.to_csv('/content/drive/MyDrive/Project5/af_tot_df_ss.csv')

## Chunk 4/8

In [ ]:
#Chunk 4/8.  Estimated execute time: 1h on local drive. 30m on Google Colab.

st_time()
ch_4_df = af_df(ch_4)

Start time: 01:07:18


HTTP Error for GET to https://api.spotify.com/v1/tracks/3GledfDACsmTu92s1Faw0B returned 404 due to non existing id
HTTP Error for GET to https://api.spotify.com/v1/tracks/0quYwXqgKZCwFs7bJF80Xj returned 404 due to non existing id


time: 35min 31s (started: 2021-03-21 01:07:18 +00:00)


In [ ]:
#Success evaluation

st_time()
chunk_df_eval(ch_4,ch_4_df)

Start time: 01:43:27
For chunk_df the dateframe rows,numbers = (9966, 36)
For chunk_df the percentage of successful audio feature extraction is 99.67
time: 5.07 ms (started: 2021-03-21 01:43:27 +00:00)


In [ ]:
#Write progress to a csv so you don't lose data with runtime crash.
#'ss' = 'subsample' and corresponds to the aggregae as you go.  It will
# change to 'tot' = "total" after the last chunk is added.

#'af' = audio features, 'tot' = total, 'df' = dataframe

#Band name is "Vaso Vagal"
vaso_vagal_df = af_df(vaso_vagal)
af_tot_df_ss = pd.concat([ch_1_df]+[ch_2_df]+[ch_3_df]+[ch_4_df]+
                   [vaso_vagal_df]).reset_index()
af_tot_df_ss.drop(columns='index')

#Export as a csv.
af_tot_df_ss.to_csv('/content/drive/MyDrive/Project5/af_tot_df_ss.csv')

## Chunk 5/8

In [ ]:
#Chunk 5/8.  Estimated execute time: 1h on local drive. 30m on Google Colab.

st_time()
ch_5_df = af_df(ch_5)

Start time: 20:17:40


HTTP Error for GET to https://api.spotify.com/v1/tracks/2PCUu5xJs8mTeMGvM532Ob returned 404 due to non existing id
HTTP Error for GET to https://api.spotify.com/v1/tracks/01z76dsMSRSNUW90Q14L4B returned 404 due to non existing id


time: 35min (started: 2021-03-20 20:17:40 +00:00)


In [ ]:
#Success evaluation

st_time()
chunk_df_eval(ch_5,ch_5_df)

Start time: 01:43:31
For chunk_df the dateframe rows,numbers = (9961, 36)
For chunk_df the percentage of successful audio feature extraction is 99.62
time: 1.94 ms (started: 2021-03-21 01:43:31 +00:00)


In [ ]:
#Write progress to a csv so you don't lose data with runtime crash.
#'ss' = 'subsample' and corresponds to the aggregae as you go.  It will
# change to 'tot' = "total" after the last chunk is added.

#'af' = audio features, 'tot' = total, 'df' = dataframe

#Band name is "Vaso Vagal"
vaso_vagal_df = af_df(vaso_vagal)
af_tot_df_ss = pd.concat([ch_1_df]+[ch_2_df]+[ch_3_df]+[ch_4_df]+[ch_5_df]+
                   [vaso_vagal_df]).reset_index()
af_tot_df_ss.drop(columns='index')

#Export as a csv.
af_tot_df_ss.to_csv('/content/drive/MyDrive/Project5/af_tot_df_ss.csv')

## Chunk 6/8

In [ ]:
#Chunk 6/8.  Estimated execute time: 1h on local drive. 30m on Google Colab.

st_time()
ch_6_df = af_df(ch_6)

Start time: 20:54:28


HTTP Error for GET to https://api.spotify.com/v1/tracks/3ORDDslop6jwYs1skgoAS5 returned 404 due to non existing id


time: 35min 14s (started: 2021-03-20 20:54:28 +00:00)


In [ ]:
#Success evaluation

st_time()
chunk_df_eval(ch_6,ch_6_df)

Start time: 21:30:01
For chunk_df the dateframe rows,numbers = (9955, 36)
For chunk_df the percentage of successful audio feature extraction is 99.56
time: 2.26 ms (started: 2021-03-20 21:30:01 +00:00)


In [ ]:
#Write progress to a csv so you don't lose data with runtime crash.
#'ss' = 'subsample' and corresponds to the aggregae as you go.  It will
# change to 'tot' = "total" after the last chunk is added.

#'af' = audio features, 'tot' = total, 'df' = dataframe

#Band name is "Vaso Vagal"
vaso_vagal_df = af_df(vaso_vagal)
af_tot_df_ss = pd.concat([ch_1_df]+[ch_2_df]+[ch_3_df]+[ch_4_df]+[ch_5_df]+
                         [ch_6_df]+[vaso_vagal_df]).reset_index()
af_tot_df_ss.drop(columns='index')

#Export as a csv.
af_tot_df_ss.to_csv('/content/drive/MyDrive/Project5/af_tot_df_ss.csv')

## Chunk 7/8

In [ ]:
#Chunk 7/8.  Estimated execute time: 1h on local drive. 30m on Google Colab.

st_time()
ch_7_df = af_df(ch_7)

Start time: 21:30:22


HTTP Error for GET to https://api.spotify.com/v1/tracks/4w3gCQjeCj0mwpIRUnL7gk returned 404 due to non existing id
HTTP Error for GET to https://api.spotify.com/v1/tracks/0ufaVne0slHc9dE28EciFC returned 404 due to non existing id


time: 35min 3s (started: 2021-03-20 21:30:22 +00:00)


In [ ]:
#Success evaluation

st_time()
chunk_df_eval(ch_7,ch_7_df)

Start time: 22:05:33
For chunk_df the dateframe rows,numbers = (9961, 36)
For chunk_df the percentage of successful audio feature extraction is 99.62
time: 3.66 ms (started: 2021-03-20 22:05:33 +00:00)


In [ ]:
#Write progress to a csv so you don't lose data with runtime crash.
#'ss' = 'subsample' and corresponds to the aggregae as you go.  It will
# change to 'tot' = "total" after the last chunk is added.

#'af' = audio features, 'tot' = total, 'df' = dataframe

#Band name is "Vaso Vagal"
vaso_vagal_df = af_df(vaso_vagal)
af_tot_df_ss = pd.concat([ch_1_df]+[ch_2_df]+[ch_3_df]+[ch_4_df]+[ch_5_df]+
                         [ch_6_df]+[ch_df_7]+[vaso_vagal_df]).reset_index()
af_tot_df_ss.drop(columns='index')

#Export as a csv.
af_tot_df_ss.to_csv('/content/drive/MyDrive/Project5/af_tot_df_ss.csv')

## Chunk 8/8

In [ ]:
#Chunk 8/8.  Estimated execute time: 1h on local drive. 30m on Google Colab.

st_time()
ch_8_df = af_df(ch_8)

Start time: 22:05:38


HTTP Error for GET to https://api.spotify.com/v1/tracks/31Ve9dl2Ti17WRKgB9QfKr returned 404 due to non existing id
HTTP Error for GET to https://api.spotify.com/v1/tracks/5szkvxcWisREix8i34QmjH returned 404 due to non existing id
HTTP Error for GET to https://api.spotify.com/v1/tracks/7DpLJaDx6Lst2ogdqEL9Ld returned 404 due to non existing id


time: 44min 42s (started: 2021-03-20 22:05:38 +00:00)


In [ ]:
#Success evaluation

st_time()
chunk_df_eval(ch_8,ch_8_df)

Start time: 01:44:45
For chunk_df the dateframe rows,numbers = (12050, 36)
For chunk_df the percentage of successful audio feature extraction is 99.54
time: 1.76 ms (started: 2021-03-21 01:44:45 +00:00)


In [ ]:
#All 8 combined in this session.
#Insert Vaso Vagal album as a tracer in a seperate file.  This is because
# I will need to do random sampling to shrink the library size down for testing.
# But I will want to tracer album to make sure I always know some songs in there
# and can do function/whatever tests using them.

af_tot_df = pd.concat([ch_1_df]+[ch_2_df]+[ch_3_df]+[ch_4_df]+
                      [ch_5_df]+[ch_6_df]+[ch_7_df]+
                      [ch_8_df]).reset_index()
af_tot_df.drop(columns='index')
af_tot_df.to_csv('/content/drive/MyDrive/Project5/af_tot_80_df.csv')

vaso_vagal=['57lYOdic1C2GVyTPZsJhlB',
            '3Rv7s6zTx6KTYZfrpBdW4C',
            '0Qj2eDY1Nku7OGymMs8HGz',
            '2pgLRrQEvDwd6jTXGDswQm',
            '5SzbfdSfxmmlyTEZcpb3EH',
            '6wUEUO6bE1R84LAB7TIAIs',
            '09u4feGdXwYsvH7hWObLPw',
            '0q4ADFP0VAFpTKyAnWuU0e',
            '7mxdqQjhmmv1iPdrE6VEmB']

vaso_vagal_df = af_df(vaso_vagal)
vaso_vagal_df.to_csv('/content/drive/MyDrive/Project5/af_vv_df.csv')

time: 5.61 s (started: 2021-03-21 02:22:59 +00:00)


# Subsample Aggregation

In [ ]:
#Exporting just a fraction for testing
#Adding my own band because duh.

#Band name is "Vaso Vagal"
vaso_vagal=['57lYOdic1C2GVyTPZsJhlB',
            '3Rv7s6zTx6KTYZfrpBdW4C',
            '0Qj2eDY1Nku7OGymMs8HGz',
            '2pgLRrQEvDwd6jTXGDswQm',
            '5SzbfdSfxmmlyTEZcpb3EH',
            '6wUEUO6bE1R84LAB7TIAIs',
            '09u4feGdXwYsvH7hWObLPw',
            '0q4ADFP0VAFpTKyAnWuU0e',
            '7mxdqQjhmmv1iPdrE6VEmB']

vaso_vagal_df = af_df(vaso_vagal)
af_tot_df_ss = pd.concat([ch_1_df]+[ch_2_df]+[ch_3_df]+[ch_4_df]+
                         [ch_5_df]+[ch_6_df]+[cd_7_df]+
                   [vaso_vagal_df]).reset_index()
af_tot_df_ss.drop(columns='index')

#Export as a csv.
af_tot_df_ss.to_csv('/content/drive/MyDrive/Project5/af_tot_df_ss.csv')

NameError: ignored

time: 1.35 s (started: 2021-03-20 20:08:49 +00:00)


In [ ]:
af_tot_df_ss


,index,track_id,track_name,album_id,album_name,artist_id,artist_name,track_preview_url,track_number,track_href,track_external_urls,track_external_id_isrc,album_href,album_ext_url,album_release_date,album_release_date_precision,album_total_tracks,album_type,album_image_large,album_image_medium,album_image_small,artist_href,artists_ext_url,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,track_length,valence
0,0,6tcTKTzid5MnguRqMXfmpq,"Concerto for Strings in A Minor, RV 161: I. Al...",1bKVopvjDzU8d115PMOy3X,Vivaldi: Concertos for Strings,2QOIawHpSlOwXDvSqQ9YJR,Antonio Vivaldi,https://p.scdn.co/mp3-preview/d6535826a85d5de8...,4,https://api.spotify.com/v1/tracks/6tcTKTzid5Mn...,https://open.spotify.com/track/6tcTKTzid5MnguR...,HKI199602304,https://api.spotify.com/v1/albums/1bKVopvjDzU8...,https://open.spotify.com/album/1bKVopvjDzU8d11...,1997-03-27,day,30,album,https://i.scdn.co/image/ab67616d0000b273d21509...,https://i.scdn.co/image/ab67616d00001e02d21509...,https://i.scdn.co/image/ab67616d00004851d21509...,https://api.spotify.com/v1/artists/2QOIawHpSlO...,https://open.spotify.com/artist/2QOIawHpSlOwXD...,0.807000,0.511,0.363,0.963000,9,0.1360,-15.223,0,44,0.0339,115.397,4,98600,0.674
1,1,3i9Tr3Vp2gjT8EsVKGt1N3,California - Chris Lake & Matroda Remix,1HFGTeIMHbKiEx1zZiDKWt,California,2zJ8chFLjiBHRNchfevMRI,SNBRN,https://p.scdn.co/mp3-preview/37ec14d1b31cf6fd...,2,https://api.spotify.com/v1/tracks/3i9Tr3Vp2gjT...,https://open.spotify.com/track/3i9Tr3Vp2gjT8Es...,QMLA71500357,https://api.spotify.com/v1/albums/1HFGTeIMHbKi...,https://open.spotify.com/album/1HFGTeIMHbKiEx1...,2015-05-11,day,2,album,https://i.scdn.co/image/ab67616d0000b2730e7de5...,https://i.scdn.co/image/ab67616d00001e020e7de5...,https://i.scdn.co/image/ab67616d000048510e7de5...,https://api.spotify.com/v1/artists/2zJ8chFLjiB...,https://open.spotify.com/artist/2zJ8chFLjiBHRN...,0.022600,0.783,0.810,0.257000,10,0.1190,-4.041,0,38,0.0794,124.028,4,305816,0.380
2,2,4Qievb8Mqy0qxdLNVl02zt,Been Caught Stealing,2Jkbi83HTSfqEd0CBdYwpU,Ritual De Lo Habitual,02NfyD6AlLA12crYzw5YcR,Jane's Addiction,https://p.scdn.co/mp3-preview/74af516b30ad8dd3...,5,https://api.spotify.com/v1/tracks/4Qievb8Mqy0q...,https://open.spotify.com/track/4Qievb8Mqy0qxdL...,USWB19000038,https://api.spotify.com/v1/albums/2Jkbi83HTSfq...,https://open.spotify.com/album/2Jkbi83HTSfqEd0...,1990-08-21,day,9,album,https://i.scdn.co/image/ab67616d0000b27380c825...,https://i.scdn.co/image/ab67616d00001e0280c825...,https://i.scdn.co/image/ab67616d0000485180c825...,https://api.spotify.com/v1/artists/02NfyD6AlLA...,https://open.spotify.com/artist/02NfyD6AlLA12c...,0.003260,0.639,0.929,0.050100,0,0.2420,-4.762,1,57,0.2120,103.758,4,214706,0.688
3,3,3dcMbvaYRwHFXnDF9CozIs,Sway (Quien Sera) - 1997 - Remastered,3M4x71xrlgVrZ2SS0Jjebq,Dino: The Essential Dean Martin,49e4v89VmlDcFCMyDv9wQ9,Dean Martin,None,5,https://api.spotify.com/v1/tracks/3dcMbvaYRwHF...,https://open.spotify.com/track/3dcMbvaYRwHFXnD...,USCA29701333,https://api.spotify.com/v1/albums/3M4x71xrlgVr...,https://open.spotify.com/album/3M4x71xrlgVrZ2S...,2004,year,30,album,https://i.scdn.co/image/ab67616d0000b2733b126b...,https://i.scdn.co/image/ab67616d00001e023b126b...,https://i.scdn.co/image/ab67616d000048513b126b...,https://api.spotify.com/v1/artists/49e4v89VmlD...,https://open.spotify.com/artist/49e4v89VmlDcFC...,0.727000,0.611,0.288,0.000002,7,0.0792,-9.988,0,0,0.0361,120.066,4,162066,0.583
4,4,4vO9dmzNRqDhFY3jD1a3P7,Mississippi Queen,2tWvZmP9EVRYqmWyJPLzjP,Climbing!,7LCp4MN0SOIVWlssid9KyE,Mountain,https://p.scdn.co/mp3-preview/a94af259aa67f896...,1,https://api.spotify.com/v1/tracks/4vO9dmzNRqDh...,https://open.spotify.com/track/4vO9dmzNRqDhFY3...,USSM10017943,https://api.spotify.com/v1/albums/2tWvZmP9EVRY...,https://open.spotify.com/album/2tWvZmP9EVRYqmW...,1970,year,10,album,https://i.scdn.co/image/ab67616d0000b273411cac...,https://i.scdn.co/image/ab67616d00001e02411cac...,https:

time: 92.2 ms (started: 2021-03-20 18:06:54 +00:00)


# Total Sample Aggregation

#### Export as csv

Note: This will go to the current directory (the one you've been working in).  The estimated size is ~1.5MB/10,000 tracks.

In [ ]:
#'af' = audio features, 'tot' = total, 'df' = dataframe

af_tot_df = pd.concat([ch_1_df]+[ch_2_df]+[ch_3_df]+
                     [ch_4_df]+[ch_5_df]+[ch_6_df]+
                     [ch_7_df]+[ch_8_df]+[vaso_vagal_df]).reset_index()
af_tot_df.drop(columns='index')

In [ ]:
af_tot_df.to_csv('af_tot_df.csv')

# Reference: Column and Audio Feature Explanation

Here is a reference for all 18 columns of the main audio feature dataframe.

In [ ]:
af_ref_df = pd.read_csv('column_af_explanation.csv')
af_tot_df

In [ ]:
#Look at a single explanation.  

#To see the audio feature type use index [0].
#To see the audio feature description use index [1]

af_ref_df['danceability'][1]

In [ ]:
plot(af_tot_df)